In [23]:
%matplotlib inline
from sklearn import datasets, linear_model
import matplotlib.pyplot as plt
import re
import seaborn as sns

import pandas as pd
import numpy as np
import datetime
import pandas as pd

In [24]:
df_airquality = pd.DataFrame.from_csv('data/airquality.csv')
df_district = pd.DataFrame.from_csv('data/district.csv')
df_meteorology = pd.DataFrame.from_csv('data/meteorology.csv')
df_station = pd.DataFrame.from_csv('data/station.csv')
df_weatherforecast = pd.DataFrame.from_csv('data/weatherforecast.csv')

In [25]:
final_df = df_airquality.join(df_station)
final_df = final_df.reset_index()
final_df['station_id'] = final_df['\ufeffstation_id']
final_df = final_df.drop('\ufeffstation_id',1)
final_df = final_df.join(df_district, on=['district_id'],rsuffix='_district')

In [26]:
df_city = pd.DataFrame.from_csv('data/city.csv')
df_city = df_city.reset_index()

df_city['city_id'] = df_city['\ufeffcity_id']
df_city['latitude_city']= df_city['longitude']
df_city['longitude_city']= df_city['longitude']

df_city = df_city.drop('latitude',1)
df_city = df_city.drop('longitude',1)

df_city = df_city.drop('\ufeffcity_id',1)
df_city['city_id'] = df_city.city_id.apply(lambda x : int(x))
final_df = pd.merge(final_df, right=df_city,on=['city_id'],how='left')


In [27]:
final_df = final_df.drop('name_chinese_x',1)
final_df = final_df.drop('name_chinese_district',1)
final_df = final_df.drop('name_chinese_y',1)
final_df = final_df.drop('name_english_y',1)

In [28]:
df_meteorology = df_meteorology.reset_index()
df_meteorology['id'] = df_meteorology['\ufeffid']
df_meteorology = df_meteorology.drop('\ufeffid',1)

In [29]:
df_meteorology['district_id'] = df_meteorology['id']
df_meteorology = df_meteorology.drop('id',1)
final_df = pd.merge(final_df, right=df_meteorology,on=['district_id','time'],how='left')

In [30]:
final_df.columns

Index(['time', 'PM25_Concentration', 'PM10_Concentration', 'NO2_Concentration',
       'CO_Concentration', 'O3_Concentration', 'SO2_Concentration',
       'name_english_x', 'latitude', 'longitude', 'district_id', 'station_id',
       'name_english_district', 'city_id', 'cluster_id', 'latitude_city',
       'longitude_city', 'weather', 'temperature', 'pressure', 'humidity',
       'wind_speed', 'wind_direction'],
      dtype='object')

In [31]:
final_df.to_csv('final_dataframe.csv')

In [2]:
final_df = pd.DataFrame.from_csv('final_dataframe.csv')
final_df['time'] = final_df.time.apply(lambda x : datetime.datetime.strptime(str(x),'%Y-%m-%d %H:%M:%S'))

In [17]:
final_df.columns

Index(['time', 'PM25_Concentration', 'PM10_Concentration', 'NO2_Concentration',
       'CO_Concentration', 'O3_Concentration', 'SO2_Concentration',
       'name_english_x', 'latitude', 'longitude', 'district_id', 'station_id',
       'name_english_district', 'city_id', 'cluster_id', 'latitude_city',
       'longitude_city', 'weather', 'temperature', 'pressure', 'humidity',
       'wind_speed', 'wind_direction'],
      dtype='object')

In [18]:
final_df.head()

,time,PM25_Concentration,PM10_Concentration,NO2_Concentration,CO_Concentration,O3_Concentration,SO2_Concentration,name_english_x,latitude,longitude,...,city_id,cluster_id,latitude_city,longitude_city,weather,temperature,pressure,humidity,wind_speed,wind_direction
0,2014-05-01 00:00:00,138.0,159.4,56.3,0.9,50.8,17.2,HaiDianBeiBuXinQu,40.090679,116.173553,...,1,1,116.407394,116.407394,0.0,20.0,1004.0,56.0,7.92,13.0
1,2014-05-01 01:00:00,124.0,163.9,38.7,0.9,51.1,17.9,HaiDianBeiBuXinQu,40.090679,116.173553,...,1,1,116.407394,116.407394,0.0,18.0,1004.0,64.0,7.56,13.0
2,2014-05-01 02:00:00,127.0,148.4,55.6,1.0,27.2,16.6,HaiDianBeiBuXinQu,40.090679,116.173553,...,1,1,116.407394,116.407394,0.0,18.0,1004.0,70.0,5.76,13.0
3,2014-05-01 03:00:00,129.0,145.6,65.7,1.0,9.7,16.7,HaiDianBeiBuXinQu,40.090679,116.173553,...,1,1,116.407394,116.407394,0.0,17.0,1004.0,74.0,6.12,13.0
4,2014-05-01 04:00:00,119.0,119.3,66.9,1.0,2.0,16.5,HaiDianBeiBuXinQu,40.090679,116.173553,...,1,1,116.407394,116.407394,0.0,17.0,1004.0,75.0,4.68,1.0


### Trying simple logistic regression for district predictons 

In [19]:
train_x = ['time',  'name_english_x', 'latitude', 'longitude', 'district_id', 'station_id', 'name_english_district', 'city_id', 'cluster_id', 'latitude_city', 'longitude_city']           
train_y_pollution = ['PM25_Concentration', 'PM10_Concentration', 'NO2_Concentration', 'CO_Concentration', 'O3_Concentration', 'SO2_Concentration']
train_y_weather = ['weather', 'temperature', 'pressure', 'humidity','wind_speed', 'wind_direction']

In [9]:
final_df[['time']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2891393 entries, 0 to 2891392
Data columns (total 1 columns):
time    datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 44.1 MB


In [12]:
partition = int(len(final_df.time) / 2)
time = final_df.time[partition]
print(time)

2015-02-03 05:00:00


In [13]:
df_test = final_df[final_df.time > time ]
df_train = final_df[final_df.time <= time ]

In [22]:
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(df_train[train_x], df_train[train_y_pollution])

In [21]:

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % np.mean((regr.predict(diabetes_X_test) - diabetes_y_test) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(diabetes_X_test, diabetes_y_test))

# Plot outputs
plt.scatter(diabetes_X_test, diabetes_y_test,  color='black')
plt.plot(diabetes_X_test, regr.predict(diabetes_X_test), color='blue',
         linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()